# 01 - Fusion des données (BAAC 2024)
---
Ce notebook a pour objectif de charger les données brutes de la sécurité routière française pour l'année 2024. 
Nous allons fusionner les tables principales pour créer un jeu de données consolidé prêt pour l'analyse.

**Données utilisées :**
- `usagers-2024.csv` : Informations sur les personnes impliquées.
- `caract-2024.csv` : Contexte de l'accident (météo, lieu, date).


In [3]:
import pandas as pd

# Chemins relatifs vers tes fichiers
path_usagers = "../data/usagers-2024.csv"
path_carac = "../data/caract-2024.csv"

# Chargement 
usagers = pd.read_csv(path_usagers, sep=";", encoding="UTF-8")
carac = pd.read_csv(path_carac, sep=";", encoding="UTF-8")

print(f"Nombre d'usagers : {usagers.shape[0]}")
print(f"Nombre d'accidents (caractéristiques) : {carac.shape[0]}")

Nombre d'usagers : 125187
Nombre d'accidents (caractéristiques) : 54402


In [4]:
# On fusionne les deux tables sur l'ID de l'accident
df = pd.merge(usagers, carac, on="Num_Acc", how="inner")

df.head()

,Num_Acc,id_usager,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,...,lum,dep,com,agg,int,atm,col,adr,lat,long
0,202400000001,203 988 581,155 781 758,A01,1,1,3,1,2003.0,2,...,2,70,70285,1,1,5,1,D438,"47,56277000","6,75832000"
1,202400000001,203 988 582,155 781 759,B01,1,1,1,1,1997.0,4,...,2,70,70285,1,1,5,1,D438,"47,56277000","6,75832000"
2,202400000002,203 988 579,155 781 757,A01,10,3,3,2,1927.0,5,...,1,21,21054,2,3,7,6,HOTEL DIEU (RUE DE L'),"47,02109000","4,83755000"
3,202400000002,203 988 580,155 781 757,A01,1,1,1,1,1987.0,4,...,1,21,21054,2,3,7,6,HOTEL DIEU (RUE DE L'),"47,02109000","4,83755000"
4,202400000003,203 988 574,155 781 756,A01,2,2,4,2,2007.0,5,...,2,15,15012,1,1,1,6,Allée des Tilleuls,"44,90238400","2,49641800"


## 2. Nettoyage des Données
---
### 2.1. Suppression des Doublons
La première étape de qualité consiste à s'assurer de l'unicité des enregistrements. Un doublon pourrait fausser les statistiques descriptives et le futur modèle d'apprentissage.

In [5]:
# Vérification initiale
initial_shape = df.shape
print(f"Dimensions avant nettoyage : {initial_shape}")

# Suppression des lignes strictement identiques
df = df.drop_duplicates()

print(f"Doublons supprimés : {initial_shape[0] - df.shape[0]}")

Dimensions avant nettoyage : (125187, 30)
Doublons supprimés : 0


### 2.2. Gestion des Valeurs Manquantes ($NaN$)
Nous analysons la proportion de données manquantes par colonne. 
Le taux de vacuité est calculé selon la formule :
$$Taux = \frac{\sum \text{Valeurs Manquantes}}{\text{Nombre total de lignes}} \times 100$$

<div class="alert alert-block alert-info">
<b>Stratégie adoptée :</b> 
1. Suppression des colonnes ayant plus de 20% de données manquantes.
2. Suppression des lignes restantes contenant des NaN pour garantir un dataset complet.
</div>

In [6]:
# Calcul du pourcentage de valeurs manquantes
missing_pct = df.isnull().sum() / len(df) * 100

# Affichage des colonnes problématiques
print("Top 10 des colonnes avec des NaN (%) :")
print(missing_pct[missing_pct > 0].sort_values(ascending=False).head(10))

# 1. Suppression des colonnes avec > 20% de vide
cols_to_drop = missing_pct[missing_pct > 20].index
df = df.drop(columns=cols_to_drop)

# 2. Suppression des lignes restantes avec des NaN
df = df.dropna()

print(f"Dimensions après traitement des NaN : {df.shape}")

Top 10 des colonnes avec des NaN (%) :
adr        4.536414
an_nais    2.060118
dtype: float64
Dimensions après traitement des NaN : (117018, 30)


### 2.3. Filtrage des Variables
Pour optimiser la performance du modèle et éviter le "bruit" informatique, nous retirons :
* **Les identifiants uniques** : Non prédictifs.
* **Les données textuelles brutes** : Nécessitent un traitement spécifique.
* **Les variables redondantes** : Pour éviter la fuite de données.

In [7]:
# Liste des colonnes à exclure pour l'analyse
# Note : on garde 'target' mais on enlèvera 'grav' plus tard pour ne pas biaiser le modèle
cols_noise = ['Num_Acc', 'adr', 'id_vehicule', 'id_usager']

# Suppression sécurisée (vérifie si la colonne existe avant de supprimer)
df = df.drop(columns=[c for c in cols_noise if c in df.columns])

print(f"Colonnes conservées : {len(df.columns)}")

Colonnes conservées : 26


### 2.4. Correction des Types de Données
Certaines variables numériques sont en réalité des catégories. Nous les convertissons au type `category` pour :
1. Améliorer la gestion de la mémoire.
2. Faciliter les futures visualisations et l'encodage du modèle.

In [8]:
# Identification des variables catégorielles (basé sur le dictionnaire BAAC)
cat_features = ['lum', 'agg', 'int', 'atm', 'col', 'catu', 'sexe', 'trajet']

for col in cat_features:
    if col in df.columns:
        df[col] = df[col].astype('category')

print("Vérification des types de données :")
print(df.dtypes.value_counts())

Vérification des types de données :
int64       10
object       7
category     2
category     1
float64      1
category     1
category     1
category     1
category     1
category     1
Name: count, dtype: int64


In [9]:
# Sauvegarde finale du dataset de travail
# index=False évite de créer une colonne inutile 'Unnamed: 0'
df.to_csv("../data/accidents_2024_final_clean.csv", index=False)